In [1]:
import pandas as pd

file_path = '../../../data/processed/ohe_25K_tracks_features_and_labels_all.csv'

tracks_features_and_labels_all_df = pd.read_csv(file_path)
tracks_features_and_labels_all_df

FileNotFoundError: ignored

In [ ]:
import random
# Set the random seed at the beginning of your code
random.seed(42)

import warnings
# Filter out all FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
grouped_df = tracks_features_and_labels_all_df.groupby("track_id").mean()
grouped_df
grouped_df_88 = grouped_df.drop('Unnamed: 0',axis=1)
grouped_df_88

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
def get_normalize():
    np.random.seed(42)
    result = None
    result = StandardScaler().fit_transform(grouped_df_88)
    return result

In [ ]:
feature_name = grouped_df_88.columns

In [ ]:
feature_name

In [ ]:
normalize_df = pd.DataFrame(get_normalize(), columns=feature_name)
correlation_matrix = normalize_df.corr()
correlation_matrix

In [ ]:
# Set display options for all columns
pd.set_option('display.max_columns', None)

# Set display options for all rows
pd.set_option('display.max_rows', None)

# Set precision for decimal points
pd.set_option('display.precision', 2)

# # Print the complete correlation matrix
# display(correlation_matrix)
# Select rows where correlation is greater than 0.05 or less than -0.05
filtered_rows = correlation_matrix[((correlation_matrix >= 0.6) | (correlation_matrix <= -0.6)) & (abs(correlation_matrix) != 1.0)]
# Print the filtered correlation matrix
total_values = len(filtered_rows.index) * len(filtered_rows.columns)
count = 0

strong_correlations = []

for x_name in filtered_rows.index:
    for y_name in filtered_rows.columns:
        value = correlation_matrix.loc[x_name, y_name]
        if (value >= 0.83 or value <= -0.83) and abs(value) != 1.0:
            strong_correlations.append((x_name, y_name, value))
            count += 1

# Sort the list of strong correlations based on the absolute correlation values
strong_correlations.sort(key=lambda x: abs(x[2]), reverse=True)

# Print the top 10 strongest correlated feature pairs
print("Top 10 Strongest Correlated Feature Pairs:")
for i, (x_name, y_name, value) in enumerate(strong_correlations[:10], 1):
    print(f"{i}. Feature X: {x_name}, Feature Y: {y_name}, Value: {value:.3f}%")

# Calculate and print the total number of strong correlation values and the percentage
percentage = (count / total_values) * 100
print(f"\nTotal number of strong correlation values: {count} (0.83-1.0: Strong correlation)")
print(f"Percentage of Total values: {percentage:.2f}%")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming correlation_matrix1 and correlation_matrix2 are your correlation matrices
axes = sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".2f")
axes.set_title('Correlation Heatmap with 88 features')

# Adjust layout to prevent overlapping titles
plt.tight_layout()

# Show the plots
plt.show()

**Finding 1:**

Deep blue color: Indicates strong negative correlation, meaning an increase in one feature is accompanied by a decrease in another feature.Deep red color: Represents strong positive correlation, implying that an increase in one feature is associated with an increase in another feature.Light color (close to white): Indicates correlation close to zero, suggesting a relatively weak linear relationship between two features.
Low correlation: Mainly light color indicates a relatively weak linear relationship between features. In other words, changes in one music feature are not likely to significantly predict changes in another feature. It can be used to avoid features affected by multicollinearity.
Nonlinear relationships: The correlation heatmap primarily displays linear relationships. If there are nonlinear relationships between features, the correlation heatmap might not capture these relationships effectively.
The top 5 pairs of features with the strongest linear correlations.
1. Feature X: spectral_centroid_mean, Feature Y: spectral_rolloff_mean, Value: 0.967%
2. Feature X: spectral_centroid_mean, Feature Y: MFCC_2_mean, Value: -0.900%
3. Feature X: spectral_rolloff_mean, Feature Y: MFCC_2_mean, Value: -0.894%
4. Feature X: spectral_centroid_var, Feature Y: spectral_rolloff_var, Value: 0.867%
5. Feature X: MFCC_16_var, Feature Y: MFCC_17_var, Value: 0.834%


In [ ]:
!pip install umap-learn==0.5.1

In [ ]:
!pip install pynndescent==0.5.4

In [ ]:
import umap
import matplotlib.pyplot as plt

# Create a UMAP model with 2 dimensions
umap_model = umap.UMAP(n_components=2, n_jobs=2, random_state=42)

# Fit UMAP without tqdm progress bar
umap_result = umap_model.fit_transform(get_normalize())

# Create a 2D scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(umap_result[:, 0], umap_result[:, 1], marker='o', s=1, cmap='viridis')

# Add labels, titles, etc.
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.title('UMAP Visualization of 88 Features')

# Show the 2D plot
plt.show()

In [ ]:
from pycaret.clustering import *

# Initialize clustering environment
exp1 = setup(umap_result,session_id=123)

# Create multiple clustering models
models = ['kmeans', 'dbscan','hclust', 'birch']

# Evaluate models and visualize clustering results
for model in models:
    # Create the clustering model
    print("\033[94m"+"\033[1m"+model)
    clustering_model = create_model(model,num_clusters=15)

In [ ]:
!pip install scikit-learn==0.24.2

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

# Define a list of candidate cluster values
k_values = [2,3,4,5,6,7,8,9, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100]

# Initialize variables for tracking the best clustering
best_calinski_score = -1
best_davies_bouldin_score = float("inf")
best_k = None

# Lists to store Calinski-Harabasz and Davies-Bouldin scores for each k
calinski_scores_list = []
davies_bouldin_scores_list = []

# Iterate over different cluster values
for k in k_values:
    # Set a random seed for reproducibility
    np.random.seed(42)

    # Create a KMeans model with 'k' clusters
    kmeans = KMeans(n_clusters=k, random_state=42)

    # Fit the model and predict cluster labels on UMAP-reduced data
    kmeans_labels = kmeans.fit_predict(umap_result)

    # Calculate Calinski-Harabasz score
    calinski_score = calinski_harabasz_score(umap_result, kmeans_labels)
    calinski_scores_list.append(calinski_score)

    # Calculate Davies-Bouldin score
    davies_bouldin = davies_bouldin_score(umap_result, kmeans_labels)
    davies_bouldin_scores_list.append(davies_bouldin)

    # Update the best clustering if a better combination is found
    if calinski_score > best_calinski_score and davies_bouldin < best_davies_bouldin_score:
        best_calinski_score = calinski_score
        best_davies_bouldin_score = davies_bouldin
        best_k = k

# Print the results of the best clustering
print("\033[94m"+"\033[1m"+f"Best number of clusters: {best_k}")
print("\033[94m"+"\033[1m"+f"Best Calinski-Harabasz score: {best_calinski_score}")
print("\033[94m"+"\033[1m"+f"Best Davies-Bouldin score: {best_davies_bouldin_score}")

# Create a figure for plotting
plt.figure(figsize=(15, 5))

# First subplot: Calinski-Harabasz Score
plt.subplot(2, 1, 1)  # 2 rows, 1 column, subplot 1
plt.plot(k_values, calinski_scores_list, marker='o', color='blue')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Calinski-Harabasz Score')
plt.title('Calinski-Harabasz Scores vs. Number of UMAP-Reduced Data Kmeans-Clusters with 88 features')
plt.grid(True)
plt.xticks(k_values)

# Second subplot: Davies-Bouldin Score
plt.subplot(2, 1, 2)  # 2 rows, 1 column, subplot 2
plt.plot(k_values, davies_bouldin_scores_list, marker='o', color='red')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Davies-Bouldin Score')
plt.title('Davies-Bouldin Score vs. Number of UMAP-Reduced Data Kmeans-Clusters with 88 features')
plt.grid(True)

# Set x-axis tick positions to the values in k_values
plt.xticks(k_values)

# Adjust layout to prevent subplot overlap
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

eps_values = [0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9]
min_samples_values = [5,10, 15, 20]

best_calinski_score = -1
best_davies_bouldin_score = float("inf")
best_eps = None
best_min_samples = None

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        dbscan_labels = dbscan.fit_predict(umap_result)

        calinski = calinski_harabasz_score(umap_result, dbscan_labels)
        davies_bouldin = davies_bouldin_score(umap_result, dbscan_labels)

        if calinski > best_calinski_score and davies_bouldin < best_davies_bouldin_score:

            best_calinski_score = calinski
            best_davies_bouldin_score = davies_bouldin
            best_eps = eps
            best_min_samples = min_samples

print("\033[94m"+"\033[1m"+"Best eps:", best_eps)
print("\033[94m"+"\033[1m"+"Best min_samples:", best_min_samples)

print("\033[94m"+"\033[1m"+"Best Calinski-Harabasz Score:", best_calinski_score)
print("\033[94m"+"\033[1m"+"Best Davies-Bouldin Score:", best_davies_bouldin_score)


In [ ]:
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

dbscan = DBSCAN(eps=best_eps, min_samples= best_min_samples)

dbscan_labels = dbscan.fit_predict(umap_result)

# Calculate the number of clusters (excluding noise points)
num_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
print("\033[94m"+"\033[1m"+"Number of cluster is:",num_clusters)
# Set colors for each cluster, including black for noise points
colors = plt.cm.viridis(np.linspace(0, 1, num_clusters + 1))
colors = np.vstack((colors, [0, 0, 0, 1]))  # Add black color for noise points

# Plot the scatter plot
plt.figure(figsize=(8, 6))
scatter = plt.scatter(umap_result[:, 0], umap_result[:, 1], c=dbscan_labels, cmap='viridis', s=1)

# Add legend outside the plot
legend_labels = list(range(num_clusters)) + ['Noise']
legend_labels = [str(label) for label in legend_labels]
plt.legend(handles=scatter.legend_elements()[0], title='Clusters', labels=legend_labels, bbox_to_anchor=(1.05, 1), loc='upper left')

# Add axis labels and title
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.title('DBSCAN Clustering of UMAP-Reduced Data with 88 features')

# Show the plot
plt.show()

Secondly, Apply PCA to deduce the dimension

In [ ]:
#it typically used to help determine how many principal components (dimensions) to retain in Principal Component Analysis (PCA).
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
pca=None
pca = PCA(random_state=42).fit(get_normalize())
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = explained_variance_ratio.cumsum()
plt.plot(range(1, len(cumulative_variance_ratio) + 1), cumulative_variance_ratio, marker='o', linestyle='--')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Variance Explained')
plt.title('Cumulative Variance Explained by Principal Components with 88 features')
plt.grid()
plt.show()

In [ ]:
np.random.seed(42)
cumulative_variance_ratio = explained_variance_ratio.cumsum()
desired_variance_ratio = 0.8  #  set 80% as cumulative variance explained threshold
components_needed = np.where(cumulative_variance_ratio >= desired_variance_ratio)[0][0] + 1
components_needed
print("\033[94m"+"\033[1m"+"The components we need in PCA with 88 features:", components_needed)

In [ ]:
from sklearn.decomposition import PCA

np.random.seed(42)
pca = PCA(n_components=28,random_state=42)
pca_result = pca.fit_transform(get_normalize())

# get contribution from each PC
explained_variance_ratio = pca.explained_variance_ratio_
eigenvalues = pca.explained_variance_

# Calculate the variance of Principal Component 1 (PC1)
variance_pc1 = eigenvalues[0]

# Calculate the variance of Principal Component 2 (PC2)
variance_pc2 = eigenvalues[1]

# Calculate the variance of Principal Component 3 (PC3)
variance_pc3 = eigenvalues[2]

# Calculate the variance contribution ratio
total_variance = sum(eigenvalues)
variance_ratio_pc1 = variance_pc1 / total_variance
variance_ratio_pc2 = variance_pc2 / total_variance
variance_ratio_pc3 = variance_pc3 / total_variance

# Format the values with two decimal places
variance_pc1_formatted = "{:.2f}".format(variance_pc1)
variance_pc2_formatted = "{:.2f}".format(variance_pc2)
variance_pc3_formatted = "{:.2f}".format(variance_pc3)
variance_ratio_pc1_formatted = "{:.2f}".format(variance_ratio_pc1)
variance_ratio_pc2_formatted = "{:.2f}".format(variance_ratio_pc2)
variance_ratio_pc3_formatted = "{:.2f}".format(variance_ratio_pc3)


print("Variance of PC1:", variance_pc1_formatted)
print("Variance of PC2:", variance_pc2_formatted)
print("Variance of PC3:", variance_pc3_formatted)
print("Variance Ratio of PC1:", variance_ratio_pc1_formatted)
print("Variance Ratio of PC2:", variance_ratio_pc2_formatted)
print("Variance Ratio of PC3:", variance_ratio_pc3_formatted)
print("\033[94m"+"\033[1m"+"Variance Ratio of PC1,PC2,PC3 with 88 features:", float(variance_ratio_pc1_formatted)+float(variance_ratio_pc2_formatted)+float(variance_ratio_pc3_formatted))

To run KMeans cluster on PCA Reduced Data

In [ ]:
import numpy as np
import umap
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score


np.random.seed(42)
pca = PCA(n_components=components_needed,random_state=42)
pca_result = pca.fit_transform(get_normalize())

k_values = [5, 6,7,8,9,10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100]

best_calinski_score = -1
best_davies_bouldin_score = float("inf")

best_k = None
calinski_scores_list = []
davies_bouldin_scores_list = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans_labels = kmeans.fit_predict(pca_result)

    calinski_score = calinski_harabasz_score(pca_result, kmeans_labels)
    calinski_scores_list.append(calinski_score)

    davies_bouldin = davies_bouldin_score(pca_result, kmeans_labels)
    davies_bouldin_scores_list.append(davies_bouldin)

    if calinski_score > best_calinski_score and davies_bouldin < best_davies_bouldin_score:
        best_calinski_score = calinski_score
        best_davies_bouldin_score = davies_bouldin
        best_k = k

print("\033[94m"+"\033[1m"+f"Best number of clusters: {best_k}")
print("\033[94m"+"\033[1m"+f"Best Calinski-Harabasz score: {best_calinski_score}")
print("\033[94m"+"\033[1m"+f"Best Davies-Bouldin score: {best_davies_bouldin_score}")

plt.figure(figsize=(15, 5))

# subplot1：Calinski-Harabasz Score
plt.subplot(2, 1, 1)
plt.plot(k_values, calinski_scores_list, marker='o', color='blue')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Calinski-Harabasz Score')
plt.title('Calinski-Harabasz Scores vs. Number of PCA-Reduced Data Kmeans-Clusters with artist name')
plt.grid(True)
plt.xticks(k_values)

# subplot2：Davies-Bouldin Score
plt.subplot(2, 1, 2)
plt.plot(k_values, davies_bouldin_scores_list, marker='o', color='red')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Davies-Bouldin Score')
plt.title('Davies-Bouldin Score vs. Number of PCA-Reduced Data Kmeans-Clusters with artist name')
plt.grid(True)
plt.xticks(k_values)
plt.tight_layout()
plt.show()


**Finding 2:**

- This may indicate that the dataset contains complex nonlinear relationships. UMAP is more proficient at capturing these nonlinear relationships during dimensionality reduction, making it more effective to use linear clustering methods on nonlinear data.PCA is primarily designed to capture linear relationships, and as a result, it may perform poorly in clustering tasks with complex nonlinear structures.

Create a UMAP model base on PCA reduced data with 2 dimensions

In [ ]:
import umap
umap_model = umap.UMAP(n_components=2, n_jobs=2, random_state=42)

# Use fit_transform to reduce the dimensionality of data in key_feature_df to 2 dimensions
umap_pca_result = umap_model.fit_transform(pca_result)

# Create a 2D scatter plot
plt.figure(figsize=(8, 6))
plt.scatter(umap_pca_result[:, 0],umap_pca_result[:, 1], marker='o', s=1, cmap='viridis')

# Add labels, titles, etc.
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.title('UMAP Visualization of Key Features with 88 features')

# Show the 2D plot
plt.show()

KMeans model based on UMAP and PCA

In [ ]:
k_values = [2,3,4,5,6,7,8,9,10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90, 100]

best_calinski_score = -1
best_davies_bouldin_score = float("inf")

best_k = None
calinski_scores_list = []
davies_bouldin_scores_list = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans_labels = kmeans.fit_predict(umap_pca_result)

    calinski_score = calinski_harabasz_score(umap_pca_result, kmeans_labels)
    calinski_scores_list.append(calinski_score)

    davies_bouldin = davies_bouldin_score(umap_pca_result, kmeans_labels)
    davies_bouldin_scores_list.append(davies_bouldin)

    if calinski_score > best_calinski_score and davies_bouldin < best_davies_bouldin_score:
        best_calinski_score = calinski_score
        best_davies_bouldin_score = davies_bouldin
        best_k = k

print("\033[94m"+"\033[1m"+f"Best number of clusters: {best_k}")
print("\033[94m"+"\033[1m"+f"Best Calinski-Harabasz score: {best_calinski_score}")
print("\033[94m"+"\033[1m"+f"Best Davies-Bouldin score: {best_davies_bouldin_score}")


plt.figure(figsize=(15, 5))

# Calinski-Harabasz Score
plt.subplot(2, 1, 1)
plt.plot(k_values, calinski_scores_list, marker='o', color='blue')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Calinski-Harabasz Score')
plt.title('Calinski-Harabasz Scores vs. Number of UMAP-Reduced Data Kmeans-Clusters with 88 features')
plt.grid(True)
plt.xticks(k_values)

# Davies-Bouldin Score
plt.subplot(2, 1, 2)
plt.plot(k_values, davies_bouldin_scores_list, marker='o', color='red')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Davies-Bouldin Score')
plt.title('Davies-Bouldin Score vs. Number of UMAP-Reduced Data Kmeans-Clusters with 88 features')
plt.grid(True)

plt.xticks(k_values)

plt.tight_layout()  # ensure subplot not overlap
plt.show()



In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

eps_values = [0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9]
min_samples_values = [5,10, 15, 20]

best_calinski_score = -1
best_davies_bouldin_score = float("inf")
best_eps = None
best_min_samples = None

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        dbscan_labels = dbscan.fit_predict(umap_pca_result)

        calinski = calinski_harabasz_score(umap_pca_result, dbscan_labels)
        davies_bouldin = davies_bouldin_score(umap_pca_result, dbscan_labels)

        if calinski > best_calinski_score and davies_bouldin < best_davies_bouldin_score:

            best_calinski_score = calinski
            best_davies_bouldin_score = davies_bouldin
            best_eps = eps
            best_min_samples = min_samples

print("\033[94m"+"\033[1m"+"Best eps:", best_eps)
print("\033[94m"+"\033[1m"+"Best min_samples:", best_min_samples)

print("\033[94m"+"\033[1m"+"Best Calinski-Harabasz Score:", best_calinski_score)
print("\033[94m"+"\033[1m"+"Best Davies-Bouldin Score:", best_davies_bouldin_score)

In [ ]:
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt

dbscan = DBSCAN(eps=0.4, min_samples= 5)

dbscan_labels = dbscan.fit_predict(umap_pca_result)

# Calculate the number of clusters (excluding noise points)
num_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
print("\033[94m"+"\033[1m"+"Number of cluster is:",num_clusters)
# Set colors for each cluster, including black for noise points
colors = plt.cm.viridis(np.linspace(0, 1, num_clusters + 1))
colors = np.vstack((colors, [0, 0, 0, 1]))  # Add black color for noise points

# Plot the scatter plot
plt.figure(figsize=(8, 6))
scatter = plt.scatter(umap_pca_result[:, 0], umap_pca_result[:, 1], c=dbscan_labels, cmap='viridis', s=1)

# Add legend outside the plot
legend_labels = list(range(num_clusters)) + ['Noise']
legend_labels = [str(label) for label in legend_labels]
plt.legend(handles=scatter.legend_elements()[0], title='Clusters', labels=legend_labels, bbox_to_anchor=(1.05, 1), loc='upper left')

# Add axis labels and title
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.title('DBSCAN Clustering of UMAP-Reduced Data with 88 features')

# Show the plot
plt.show()

**Finding 3:**


- If clustering methods are applied to data that has been jointly processed by UMAP and PCA, regardless of which clustering methodology is used, the metric test scores are significantly better than when using PCA alone to process the data.
- The original data is also applicable in cases where PCA and UMAP are jointly applied for processing.
- Especially when clustering using the DBscan method, we found that the results were even better than those obtained by processing the data with UMAP alone. Therefore, we ultimately chose the DBscan clustering method to extract cluster labels.




In [ ]:
# from pycaret.clustering import *

# # Initialize clustering environment
# exp1 = setup(umap_pca_result,session_id=123)

# # Create multiple clustering models
# models = ['kmeans', 'dbscan','hclust', 'birch']

# # Evaluate models and visualize clustering results
# for model in models:
#     # Create the clustering model
#     print("\033[94m"+"\033[1m"+model)
#     clustering_model = create_model(model,num_clusters=15)

In [ ]:
# kmeans_umap = KMeans(n_clusters=15, random_state=42)
# kmeans_umap_labels = kmeans_umap.fit_predict(umap_result)

# grouped_umap_15_88feature_df = grouped_df_88.copy()
# grouped_umap_15_88feature_df['KMeans_umap_Labels'] = kmeans_umap_labels

# grouped_umap_15_88feature_df.to_csv('grouped_umap_15_88feature_df.csv',index=True)
# grouped_umap_15_88feature_df['KMeans_umap_Labels'].value_counts()



In [ ]:
dbscan = DBSCAN(eps=0.4, min_samples=5)

dbscan_labels = dbscan.fit_predict(umap_pca_result)

grouped_dbscan_12_88feature_df = grouped_df_88.copy()
grouped_dbscan_12_88feature_df['dbscan_umap_pca_Labels'] = dbscan_labels

grouped_dbscan_12_88feature_df.to_csv('grouped_dbscan_12_88feature_df.csv',index=True)
grouped_dbscan_12_88feature_df['dbscan_umap_pca_Labels'].value_counts()

In [ ]:
grouped_dbscan_12_88feature_df

In [ ]:
# grouped_umap_15_88feature_df

In [ ]:
merged_df_dbscan = pd.merge(grouped_dbscan_12_88feature_df, tracks_features_and_labels_all_df[['track_id', 'track_genre_top']], on='track_id', how='left')

# Create a new 'target' column and fill it with 'track_genre_top' values
merged_df_dbscan['target'] = merged_df_dbscan['track_genre_top']

In [ ]:
merged_df_dbscan = merged_df_dbscan.drop('track_genre_top', axis=1)

merged_df_with_dbscan_umap_Labels = merged_df_dbscan.drop('track_id',axis=1)
# merged_df_with_KMeans_umap_Labels_without_keyscale = merged_df.drop(['track_id', 'key_A', 'key_Ab', 'key_B', 'key_Bb',
#        'key_C', 'key_C#', 'key_D', 'key_E', 'key_Eb', 'key_F', 'key_F#',
#        'key_G', 'scale_major', 'scale_minor'],axis=1)
display(merged_df_with_dbscan_umap_Labels)

merged_df_no_dbscan_umap_Labels = merged_df_with_dbscan_umap_Labels.drop('dbscan_umap_pca_Labels',axis=1)
# merged_df_no_KMeans_umap_Labels_without_keyscale = merged_df.drop(['track_id', 'key_A', 'key_Ab', 'key_B', 'key_Bb',
#        'key_C', 'key_C#', 'key_D', 'key_E', 'key_Eb', 'key_F', 'key_F#',
#        'key_G', 'scale_major', 'scale_minor','KMeans_umap_Labels'],axis=1)
merged_df_no_dbscan_umap_Labels

In [ ]:
# merged_df = pd.merge(grouped_umap_15_88feature_df, tracks_features_and_labels_all_df[['track_id', 'track_genre_top']], on='track_id', how='left')

# # Create a new 'target' column and fill it with 'track_genre_top' values
# merged_df['target'] = merged_df['track_genre_top']



# # Drop the redundant 'track_genre_top' column
# merged_df = merged_df.drop('track_genre_top', axis=1)

# merged_df_with_KMeans_umap_Labels = merged_df.drop('track_id',axis=1)
# merged_df_with_KMeans_umap_Labels_without_keyscale = merged_df.drop(['track_id', 'key_A', 'key_Ab', 'key_B', 'key_Bb',
#        'key_C', 'key_C#', 'key_D', 'key_E', 'key_Eb', 'key_F', 'key_F#',
#        'key_G', 'scale_major', 'scale_minor'],axis=1)
# display(merged_df_with_KMeans_umap_Labels)
# merged_df_no_KMeans_umap_Labels = merged_df_with_KMeans_umap_Labels.drop('KMeans_umap_Labels',axis=1)
# merged_df_no_KMeans_umap_Labels_without_keyscale = merged_df.drop(['track_id', 'key_A', 'key_Ab', 'key_B', 'key_Bb',
#        'key_C', 'key_C#', 'key_D', 'key_E', 'key_Eb', 'key_F', 'key_F#',
#        'key_G', 'scale_major', 'scale_minor','KMeans_umap_Labels'],axis=1)
# merged_df_no_KMeans_umap_Labels

In [ ]:
merged_df_dbscan.to_csv('merged_one_hot_df_dbscan_88.csv',index=False)
merged_df_dbscan

In [ ]:
# merged_df_with_KMeans_umap_Labels.to_csv('/content/drive/My Drive/Colab Notebooks/merged_df_with_KMeans_umap_Labels.csv',index=True)
# merged_df_no_KMeans_umap_Labels.to_csv('/content/drive/My Drive/Colab Notebooks/merged_df_no_KMeans_umap_Labels.csv',index=True)
merged_df.to_csv('merged_one_hot_df_88.csv',index=False)
merged_df

In [ ]:
from pycaret.classification import *

# Initialize clustering environment
exp1 = setup(data=merged_df_with_dbscan_umap_Labels,
             target="target",
             categorical_features=["dbscan_umap_pca_Labels"],
             normalize=True,
             fix_imbalance=True,
             session_id=123)


model = create_model('lightgbm',class_weight='balanced')

In [ ]:
from pycaret.classification import *


# Initialize clustering environment
exp1 = setup(data=merged_df_no_dbscan_umap_Labels,
             target="target",
             normalize=True,
             fix_imbalance=True,
             session_id=123)

model = create_model('lightgbm', class_weight='balanced')

In [ ]:
# from pycaret.classification import *

# # Initialize clustering environment
# exp1 = setup(data=merged_df_with_KMeans_umap_Labels,
#              target="target",
#              categorical_features=["KMeans_umap_Labels"],
#              normalize=True,
#              fix_imbalance=True,
#              session_id=123)


# model = create_model('lightgbm',class_weight='balanced')



In [ ]:
# from pycaret.classification import *


# # Initialize clustering environment
# exp1 = setup(data=merged_df_no_KMeans_umap_Labels,
#              target="target",
#              normalize=True,
#              fix_imbalance=True,
#              session_id=123)

# model = create_model('lightgbm', class_weight='balanced')


In [ ]:
# from pycaret.classification import *


# # Initialize clustering environment
# exp1 = setup(data=merged_df_no_KMeans_umap_Labels_without_keyscale,
#              target="target",
#              normalize=True,
#              fix_imbalance=True,
#              session_id=123)

# model = create_model('lightgbm', class_weight='balanced')

In [ ]:
# from pycaret.classification import *


# # Initialize clustering environment
# exp1 = setup(data=merged_df_with_KMeans_umap_Labels_without_keyscale,
#              target="target",
#              normalize=True,
#              fix_imbalance=True,
#              session_id=123)

# model = create_model('lightgbm', class_weight='balanced')

**Finding 4:**

- Although the experiments above show that the clustering effect of DBscan is quite good, when we use the obtained cluster labels as additional features and incorporate them into the supervised learning model training with the existing 88 features, we did not observe a significant improvement in the model's performance.
- It is possible that the clustering labels are strongly correlated with features such as "key" and "scale." Alternatively, errors in the genre types of the original data could be causing the issue.
- Therefore, in the end, we decided not to incorporate the clustering labels obtained from unsupervised learning into our dataset for supervised learning training and evaluation.